In [2]:
import json, urllib
# import plotly.plotly as py
import pandas as pd
import numpy as np
from bokeh.palettes import Set1, Category20c, Viridis, Set3, Magma256
import pylab
import seaborn as sns
import plotly.offline as ply
ply.init_notebook_mode(connected=True)

In [3]:
data = pd.read_csv('/Users/michael.chestnut/Documents/DOI/data/data.csv')

In [4]:
##Data Cleaning##
#Preparing df to be inserted in the dashboard class. 
#If this is done at scale perhaps it is better to do this in SQL?
columns = ['UII','bureau_name','portfolio_part','investment_title','fea_service_area','FY18','FY19',
           'FY20','date']
old_columns = ['Unique Investment Identifier','Bureau Name','Part Of IT Portfolio','Investment Title',
              'FEA BRM Services - Primary service area','Total IT Spending FY2018 (PY) ($ M)',
              'Total IT Spending FY2019 (CY) ($ M)','Total IT Spending FY2020 (BY) ($ M)','Updated Date']

#Putting this into a function so it can be built out as needed without getting too messy
def clean_data(df,columns,old_columns):
    df = df[old_columns]
    df.columns=columns
    return df

def rename(part):
    if part == '01 - Mission Delivery':
        return 'Mission Delivery'
    elif part == '02 - Administrative Services and Support Systems':
        return 'Admin Services'
    else:
        return 'IT'

df = clean_data(data, columns, old_columns)
df['portfolio_part'] = df['portfolio_part'].apply(rename)
df.head(5)

/Users/michael.chestnut/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,UII,bureau_name,portfolio_part,investment_title,fea_service_area,FY18,FY19,FY20,date
0,010-000002256,Department of the Interior,IT,IBC - Infrastructure - IT Security & Compliance,315 - Threat and Vulnerability Management,0.000000,0.000000,0.000000,2019-04-03
1,010-000002378,Department of the Interior,Admin Services,DOI - FOIA Portal (DOJ),319 - Regulatory Compliance,0.000000,0.030000,0.030000,2019-04-03
2,010-000001448,Department of the Interior,Mission Delivery,OS - Receipts Expenditures and Allocation Mode...,101 - Budget Formulation,0.095000,0.103000,0.101000,2019-04-03
3,010-000000310,Department of the Interior,Admin Services,DOI - Enterprise Facilities Maintenance & Mana...,129 - Reporting and Information,5.175915,5.175962,5.259022,2019-04-03
4,010-000002321,Department of the Interior,Admin Services,DOI - Compliance Support ESF-11/ESF-11 Website,008 - Disaster Preparedness and Planning,0.090902,0.106702,0.090902,2019-04-03


In [105]:
def assign_colors(portfolio):
    if portfolio == 'IT':
        return '#8dd3c7'
    elif portfolio == 'Mission Delivery':
        return '#ffffb3'
    else:
        return '#bebada'

In [106]:
edges = df.groupby(['bureau_name','portfolio_part']).sum().reset_index()
N = len(edges)
colors = Magma256[1:N+1]


edges['label'] = edges['bureau_name'].map(str) + ": " + edges['portfolio_part']
edges['color'] = '#262C46'
edges['link_color'] = edges['portfolio_part'].apply(assign_colors)
edges.head(5)

,bureau_name,portfolio_part,FY18,FY19,FY20,label,color,link_color
0,Bureau of Indian Affairs and Bureau of Indian ...,IT,61.422675,55.291045,56.111237,Bureau of Indian Affairs and Bureau of Indian ...,#262C46,#8dd3c7
1,Bureau of Indian Affairs and Bureau of Indian ...,Mission Delivery,19.017266,20.756476,20.162536,Bureau of Indian Affairs and Bureau of Indian ...,#262C46,#ffffb3
2,Bureau of Land Management,IT,49.481013,55.241222,63.650184,Bureau of Land Management: IT,#262C46,#8dd3c7
3,Bureau of Land Management,Mission Delivery,22.495620,35.408510,32.930503,Bureau of Land Management: Mission Delivery,#262C46,#ffffb3
4,Bureau of Ocean Energy Management,IT,0.000000,0.000000,0.000000,Bureau of Ocean Energy Management: IT,#262C46,#8dd3c7


In [107]:
def plot_sankey(df, fy='FY18'):
    filtered_df = df[['bureau_name','portfolio_part','color','label','link_color',fy]]
    
    
    portfolio=filtered_df['portfolio_part'].tolist()
    bureau=filtered_df['bureau_name'].tolist()
    
    unique_values=pd.unique(pd.concat((filtered_df['bureau_name'],filtered_df['portfolio_part']),axis=0)).tolist()
    
    portfolio_indices=[unique_values.index(i) for i in portfolio]
    print(len(portfolio_indices))
   
    bureau_indices=[unique_values.index(i) for i in bureau]
#     traces = []
    
    data_trace = dict(
        type='sankey',
        domain=dict(
            x=[0,1],
            y=[0,1]
        ),
        orientation="h",
        valueformat=".0f",
        node=dict(
            pad = 10,
            thickness=30,
            line=dict(
                color="black",
                width=0.5
            ),
            label= unique_values,
            color = filtered_df['color']
        ),
        link=dict(
            source=bureau_indices,
            target=portfolio_indices,
            value=filtered_df[fy].dropna(axis=0,how='any'),
            color=filtered_df['link_color']
        )
    )
    layout= dict(
        title="Bureau Investments by Portfolio Part",
        height=700,
        width=900,
        font=dict(
            size=10
        ),
    )

    fig = dict(data=[data_trace],layout=layout)
    ply.iplot(fig,filename='practice-sankey',validate=False)
    

In [108]:
plot_sankey(edges)

33


In [27]:
data = pd.read_csv('/Users/michael.chestnut/Documents/DOI/data/cleaned_data.csv')
df = data.drop(columns='Unnamed: 0')
df.head(5)

,UII,bureau_name,portfolio_part,investment_title,fea_service_area,FY18,FY19,FY20,date
0,010-000002256,Department of the Interior,IT,IBC - Infrastructure - IT Security & Compliance,315 - Threat and Vulnerability Management,0.000000,0.000000,0.000000,2019-04-03
1,010-000002378,Department of the Interior,Admin Services,DOI - FOIA Portal (DOJ),319 - Regulatory Compliance,0.000000,0.030000,0.030000,2019-04-03
2,010-000001448,Department of the Interior,Mission Delivery,OS - Receipts Expenditures and Allocation Mode...,101 - Budget Formulation,0.095000,0.103000,0.101000,2019-04-03
3,010-000000310,Department of the Interior,Admin Services,DOI - Enterprise Facilities Maintenance & Mana...,129 - Reporting and Information,5.175915,5.175962,5.259022,2019-04-03
4,010-000002321,Department of the Interior,Admin Services,DOI - Compliance Support ESF-11/ESF-11 Website,008 - Disaster Preparedness and Planning,0.090902,0.106702,0.090902,2019-04-03


In [32]:
bureaus= pd.unique(df['bureau_name']).tolist()

In [38]:
abbrevs = ['DOI', 'BLM', 'BOEM', 'OSMRE', 'BR', 'USGS', 'FWS', 'BSEE', 'NPS', 'BIA','DO','SOL','OST']

In [39]:
dicty={}
for x,y in zip(bureaus,abbrevs):
    dicty[x]=y
dicty

{'Department of the Interior': 'DOI',
 'Bureau of Land Management': 'BLM',
 'Bureau of Ocean Energy Management': 'BOEM',
 'Office of Surface Mining Reclamation and Enforcement': 'OSMRE',
 'Bureau of Reclamation': 'BR',
 'United States Geological Survey': 'USGS',
 'United States Fish and Wildlife Service': 'FWS',
 'Bureau of Safety and Environmental Enforcement': 'BSEE',
 'National Park Service': 'NPS',
 'Bureau of Indian Affairs and Bureau of Indian Education': 'BIA',
 'Departmental Offices': 'DO',
 'Office of the Solicitor': 'SOL',
 'Office of the Special Trustee for American Indians': 'OST'}

In [40]:
def assign_abbrev(bureau,dicty):
    return dicty[bureau]

In [37]:
assign_abbrev('Bureau of Land Management',dicty)

'BLM'